In [3]:
import multiprocessing as mp
import time
import socket
import os

In [3]:
mp.cpu_count()

12

In [4]:
mp.active_children()

[]

In [5]:
mp.get_all_start_methods()

['spawn', 'fork', 'forkserver']

In [6]:
mp.get_start_method()

'spawn'

In [9]:
os.getpid(), os.getppid()

(44817, 30524)

In [10]:
mp.set_start_method("fork")

RuntimeError: context has already been set

In [32]:
def print_data(val):
    print(f"print_data {os.getpid()=}", val)
    time.sleep(val)
    print("print_data after sleep")


proc = mp.Process(target=print_data, args=(99,))
proc.start()
proc.join()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=84, pipe_handle=114)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: module '__main__' has no attribute 'print_data'


In [22]:
ctx = mp.get_context("fork")

In [62]:
proc = ctx.Process(target=print_data, args=(99,))
proc.start()

print_data os.getpid()=28379 99


In [63]:
proc.daemon, proc.is_alive(), proc.name, proc.pid, mp.active_children()

(False,
 True,
 'ForkProcess-102',
 28379,
 [<ForkProcess name='ForkProcess-102' pid=28379 parent=44817 started>])

In [40]:
!ps -ef | grep 75248

/Users/g.kandaurov/.zshenv:source:1: no such file or directory: .bash_profile
  502 75248 44817   0  8:27   ??         0:00.01 /Library/Frameworks/Python.framework/Versions/3.14/Resources/Python.app/Contents/MacOS/Python -m ipykernel_launcher -f /Users/g.kandaurov/Library/Jupyter/runtime/kernel-d7a200b4-0298-49b2-99dd-b8fa5b98a8a5.json
  502 76084 44817   0  8:27   ttys012    0:00.01 /bin/zsh -c ps -ef | grep 75248
  502 76086 76084   0  8:27   ttys012    0:00.00 grep 75248


In [64]:
!kill 9 28379

/Users/g.kandaurov/.zshenv:source:1: no such file or directory: .bash_profile
zsh:kill:1: kill 9 failed: no such process


In [65]:
proc.daemon, proc.is_alive(), proc.name, proc.pid

(False, False, 'ForkProcess-102', 28379)

In [15]:
N = 10 ** 9


def counter(a, b):
    print(f"start {os.getpid()=}, {os.getppid()=}", a, b)
    while a < b:
        a += 1
    print(f"finish {os.getpid()=}, {os.getppid()=}", a, b)

In [19]:
%%time

counter(0, N)
print("total finish")

start os.getpid()=44817, os.getppid()=30524 0 1000000000
finish os.getpid()=44817, os.getppid()=30524 1000000000 1000000000
total finish
CPU times: user 8.6 s, sys: 95.3 ms, total: 8.7 s
Wall time: 8.83 s


In [29]:
%%time

N = 10 ** 9
PROCS = 4
JOBS = N // PROCS

procs = [
    ctx.Process(target=counter, args=(i * JOBS, (i + 1) * JOBS))
    for i in range(PROCS)
]

for pr in procs:
    pr.start()

for pr in procs:
    pr.join()

print("total finish")

start os.getpid()=44153, os.getppid()=44817 start os.getpid()=44154, os.getppid()=448170start os.getpid()=44157, os.getppid()=44817  250000000  500000000250000000500000000 

750000000start os.getpid()=44158, os.getppid()=44817
 750000000 1000000000
finish os.getpid()=44157, os.getppid()=44817 750000000 750000000
finish os.getpid()=44158, os.getppid()=44817 1000000000 1000000000
finish os.getpid()=44154, os.getppid()=44817 500000000 500000000
finish os.getpid()=44153, os.getppid()=44817 250000000 250000000
total finish
CPU times: user 13.6 ms, sys: 28.4 ms, total: 42 ms
Wall time: 3.05 s


In [30]:
N = 10 ** 9


def counter(a, b, lock):
    with lock:
        print(f"start {os.getpid()=}, {os.getppid()=}", a, b)

    while a < b:
        a += 1

    with lock:
        print(f"finish {os.getpid()=}, {os.getppid()=}", a, b)

In [31]:
%%time

N = 10 ** 9
PROCS = 4
JOBS = N // PROCS
lock = mp.Lock()

procs = [
    ctx.Process(target=counter, args=(i * JOBS, (i + 1) * JOBS, lock))
    for i in range(PROCS)
]

for pr in procs:
    pr.start()

for pr in procs:
    pr.join()

print("total finish")

start os.getpid()=56825, os.getppid()=44817 0 250000000
start os.getpid()=56826, os.getppid()=44817 250000000 500000000
start os.getpid()=56827, os.getppid()=44817 500000000 750000000
start os.getpid()=56828, os.getppid()=44817 750000000 1000000000
finish os.getpid()=56826, os.getppid()=44817 500000000 500000000
finish os.getpid()=56828, os.getppid()=44817 1000000000 1000000000
finish os.getpid()=56825, os.getppid()=44817 250000000 250000000
finish os.getpid()=56827, os.getppid()=44817 750000000 750000000
total finish
CPU times: user 13.6 ms, sys: 29 ms, total: 42.7 ms
Wall time: 3.31 s


In [51]:
class User:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f"User[{self.name}, {id(self)}]"

In [58]:
def update_data(data):
    print(f"start update_data {os.getpid()}", data)

    data[123] = "123"
    data[("user", "steve", 11)] = User("steve")
    data["to_update"] = "updated"
    
    print(f"finish update_data {os.getpid()}", data)

In [59]:
data = {"to_update": "init"}

update_data(data)

print("after update", data)

start update_data 44817 {'to_update': 'init'}
finish update_data 44817 {'to_update': 'updated', 123: '123', ('user', 'steve', 11): User[steve, 4472747152]}
after update {'to_update': 'updated', 123: '123', ('user', 'steve', 11): User[steve, 4472747152]}


In [61]:
data = {"to_update": "init"}
print("before", data)

proc = ctx.Process(target=update_data, args=(data,))
proc.start()
proc.join()

print("after", data)

before {'to_update': 'init'}
start update_data 20086 {'to_update': 'init'}
finish update_data 20086 {'to_update': 'updated', 123: '123', ('user', 'steve', 11): User[steve, 4490595984]}
after {'to_update': 'init'}


In [4]:
ctx = mp.get_context("fork")

In [10]:
class User:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f"User[{self.name}, {id(self)}]"


def update_data(data):
    print(f"start update_data {os.getpid()}", data)

    data[123] = "123"
    data[("user", "steve", 11)] = User("steve")
    data["to_update"] = "updated"
    
    print(f"finish update_data {os.getpid()}", data)


with ctx.Manager() as manager:
    data = manager.dict()
    data["to_update"] = "init"
    print("before", data)

    proc = ctx.Process(target=update_data, args=(data,))
    proc.start()
    proc.join()

    print("after", os.getpid(), data)
    backup = dict(data)

print("total end", backup)

before {'to_update': 'init'}
start update_data 81472 {'to_update': 'init'}
finish update_data 81472 {'to_update': 'updated', 123: '123', ('user', 'steve', 11): User[steve, 4739035360]}
after 57960 {'to_update': 'updated', 123: '123', ('user', 'steve', 11): User[steve, 4739035360]}
total end {'to_update': 'updated', 123: '123', ('user', 'steve', 11): User[steve, 4739033680]}


In [7]:
os.getpid()

57960

In [26]:
def client(data):
    print(f"start client with {data=}")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect(("localhost", 25_001))
        sock.sendall(data.encode())
        print("client: data sent")
        print("client: recv", sock.recv(1024).decode())

    print(f"finish client")


def server():
    print(f"start server")
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)
        sock.bind(("localhost", 25_001))
        sock.listen(5)

        i = 0
        while i < 5:
            i += 1
            client, addr = sock.accept()
            print("server: conn from", addr, client)
    
            with client:
                data = client.recv(1024).decode()
                print("server: recv", data)
                client.sendall(data.upper().encode())

    print("finish server")

In [27]:
proc_server = ctx.Process(target=server)
proc_server.start()

start server
server: conn from ('127.0.0.1', 62615) <socket.socket fd=36, family=2, type=1, proto=0, laddr=('127.0.0.1', 25001), raddr=('127.0.0.1', 62615)>
server: recv Варенье из картошки
server: conn from ('127.0.0.1', 62648) <socket.socket fd=36, family=2, type=1, proto=0, laddr=('127.0.0.1', 25001), raddr=('127.0.0.1', 62648)>
server: recv Варенье из картошки
server: conn from ('127.0.0.1', 62689) <socket.socket fd=36, family=2, type=1, proto=0, laddr=('127.0.0.1', 25001), raddr=('127.0.0.1', 62689)>
server: recv лампа
server: conn from ('127.0.0.1', 62720) <socket.socket fd=36, family=2, type=1, proto=0, laddr=('127.0.0.1', 25001), raddr=('127.0.0.1', 62720)>
server: recv лампа1
server: conn from ('127.0.0.1', 62751) <socket.socket fd=36, family=2, type=1, proto=0, laddr=('127.0.0.1', 25001), raddr=('127.0.0.1', 62751)>
server: recv лампа
finish server


In [28]:
proc_client = ctx.Process(target=client, args=("Варенье из картошки",))
proc_client.start()
proc_client.join()

start client with data='Варенье из картошки'
client: data sent
client: recv ВАРЕНЬЕ ИЗ КАРТОШКИ
finish client


In [17]:
proc_server.join()

In [29]:
proc_server.is_alive()

True

In [31]:
proc_client = ctx.Process(target=client, args=("лампа",))
proc_client.start()
proc_client.join()

start client with data='лампа'
client: data sent
client: recv ЛАМПА
finish client


In [32]:
proc_client = ctx.Process(target=client, args=("лампа1",))
proc_client.start()
proc_client.join()

start client with data='лампа1'
client: data sent
client: recv ЛАМПА1
finish client


In [33]:
proc_client = ctx.Process(target=client, args=("лампа",))
proc_client.start()
proc_client.join()

start client with data='лампа'
client: data sent
client: recv ЛАМПА
finish client


In [34]:
proc_client = ctx.Process(target=client, args=("лампа",))
proc_client.start()
proc_client.join()

start client with data='лампа'


Process ForkProcess-29:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/multiprocessing/process.py", line 320, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/j3/m947cq0j0fx8t0rrdzw5kmb00000gp/T/ipykernel_57960/1524470039.py", line 4, in client
    sock.connect(("localhost", 25_001))
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
ConnectionRefusedError: [Errno 61] Connection refused
